In [370]:
from collections import deque
from enum import Enum
import graphviz as gv
import random

In [371]:
class char(Enum):
    VAR = 1
    SIGN = 2
def char_ind(a):
    return ord(a) - ord('a') if ord(a) > 90 else 26 + ord(a) - ord('A')
def ind_char(a):
    if a > 25:
        return chr(65 + a - 26)
    return chr(97 + a)

global count
count = 0
appeared = [-1] * 52
vert_to_char = [-1] * 52
f1 = open("input.txt", "r")
f2 = open("input_version2.txt", "r")

In [372]:
def process_string(res, prod1, appeared1, vert_to_char1, nr):
    global count

    first_char = ord(res[0]) - ord('a')
    if appeared1[first_char] == -1:
        appeared1[first_char] = count
        vert_to_char1[count] = first_char
        prod1[count][0].append(nr)
        count += 1
    else:
        prod1[appeared1[first_char]][0].append(nr)

    ind = 4
    state = char.VAR
    signs = ['+', '-', '*', '/', '^', '%']

    while ind < len(res):
        while ind < len(res) and (res[ind] > 'z' or res[ind] < 'A' or 'Z' < res[ind] < 'a'):
            if res[ind] in signs:
                if state == char.VAR:
                    raise Exception("Bledne dane")
                state = char.VAR
            elif '0' <= res[ind] <= '9':
                if state == char.SIGN:
                    raise Exception("Bledne dane")
                while ind < len(res) and '0' <= res[ind] <= '9':
                    ind += 1
                state = char.SIGN
                continue
            ind += 1

        if ind < len(res):
            if state == char.SIGN:
                raise Exception("Bledne dane")
            other_char = char_ind(res[ind])
            if appeared1[other_char] == -1:
                appeared1[other_char] = count
                prod1[count][1].append(nr)
                count += 1
            else:
                prod1[appeared1[other_char]][1].append(nr)
            state = char.SIGN
        ind += 1
    if state == char.VAR:
        raise Exception("Bledne dane")



In [373]:
def process_input_v1(f):
    global count
    no_lines = int(f.readline().strip('\n'))
    #print(no_lines)

    D = [[0 for _ in range(no_lines)] for _ in range(no_lines)]
    for i in range(no_lines):
        D[i][i] = 1


    prod = [[[], []] for _ in range(52)]
    for i in range(no_lines):
        res_i = f.readline().strip('\n')
        #print(res_i)
        process_string(res_i, prod, appeared, vert_to_char, i)

    for i in range(count):
        for l in prod[i][0]:
            for r in prod[i][1]:
                D[l][r] = 1
                D[r][l] = 1
                
    ver_list = f.readline().strip('\n')
    #print(ver_list)
    f.close()
    return D, ver_list


def process_input_v2(f):

    no_prod = int(f.readline().strip('\n'))
    no_var = int(f.readline().strip('\n'))
    D = [[0 for _ in range(no_prod)] for _ in range(no_prod)]
    for i in range(no_prod):
        D[i][i] = 1
    
    prod = [[[], []] for _ in range(no_var)]
    

    for j in range(no_prod):
        r = int(f.readline().strip('\n'))
        l = int(f.readline().strip('\n'))
        prod[l][0].append(j)
        R = f.readline().strip('\n').split(' ')
        if r > 0:
            for var in R:
                prod[int(var)][1].append(j)
    

    for i in range(no_var):
        for l in prod[i][0]:
            for r in prod[i][1]:
                D[l][r] = 1
                D[r][l] = 1
    
    ver_list = f.readline().strip('\n')
    #print(ver_list)
    f.close()
    return D, ver_list




In [374]:
def bfs_with_deletion(start_vertex, Graph, show = False):
    Q = deque()
    ways = [0] * len(Graph)
    Q.append(start_vertex)
    while len(Q):
        v = Q.popleft()
        ways[v] += 1
        for w, active in Graph[v]:
            if active:
                Q.append(w)

    for j in range(len(Graph[start_vertex])):
        if ways[Graph[start_vertex][j][0]] > 1:
            if show:
                print(f"Deleting edge: {start_vertex+1} -> {Graph[start_vertex][j][0]+1}")
            Graph[start_vertex][j][1] = False

In [375]:
def generate_input(fout, n = 50, m = 40, word_len = 100):
    fout.write(str(n) + '\n')
    fout.write(str(m) + '\n')
    for i in range(n):
        num = random.randint(0, m//2)
        fout.write(str(num) + '\n')
        l = random.randint(0, m-1)
        fout.write(str(l) + '\n')
        with_spaces = ""
        for j in range(num):
            if j > 0:
                with_spaces += ' ' 
            with_spaces += str(random.randint(0, m-1)) 
        fout.write(with_spaces + '\n')
    word = ""
    for i in range(word_len):
        if i > 0:
            word += ' ' 
        word += str(random.randint(0, n-1)) 
    fout.write(word + '\n')

In [376]:
example_name = "example.txt"
f_test = open("generated_inputs\\" + example_name, "w")
n1, m1 = 65, 40
generate_input(f_test, n = n1, m = m1, word_len = 24)
f_test.close()

In [377]:
Gr = gv.Digraph(format='png',
                filename='graph.gv', engine='fdp')   #['circo', 'dot', 'fdp', 'neato', 'osage', 'patchwork', 'sfdp', 'twopi']

f3 = open("generated_inputs\\" + example_name, "r")

vertexes = []
version = 2
more = (version == 2 and n1 > 52) #Zmienna more oznacza czy mamy wersje wygenerowanych danych i liczbę operacji(transakcji) większą od 52

if not version:
    D, ver_list = process_input_v1(f1)
elif version == 1:
    D, ver_list = process_input_v2(f2)
else:
    D, ver_list = process_input_v2(f3)

#print(ver_list)

if not version:
    for pos, v in enumerate(ver_list):
        vertexes.append(char_ind(v))
        Gr.node(str(pos), v)
else:
    for id, v in enumerate(ver_list.strip('\n').split(' ')):
        let = ''
        if not more:
            Gr.node(str(id), ind_char(int(v)))
            let = ind_char(int(v))
        else:
            Gr.node(str(id), v)
            let = v + ' '
        print(let, end = '')
        vertexes.append(int(v))
    print('')

#print(vertexes)

m = len(vertexes)
G = [[] for _ in range(m)]
inner_edges = [0] * m
Foata = [[] for _ in range(m)]
in_class = [0] * m

for i in range(m - 1):
    for j in range(i + 1, m):
        if D[vertexes[i]][vertexes[j]]:
            G[i].append([j, True])

for i in range(m - 1):
    bfs_with_deletion(i, G)

#print("Graph:")

for i in range(m):
    for v, visit in G[i]:
        if visit:
            inner_edges[v] += 1
            Gr.edge(str(i), str(v), constraint='false')
            #print(f"{i+1} -> {v+1}")

for k in range(m):
    Fi = []
    found = 0
    for i in range(m):
        if not in_class[i] and inner_edges[i] == 0:
            found = 1
            in_class[i] = k + 1
            Foata[k].append(i)

    for i in range(m):
        if in_class[i] == k + 1:
            for v, visit in G[i]:
                if visit:
                    inner_edges[v] -= 1
    if not found:
        break

print("Foata classes:")
for i in range(m):
    if len(Foata[i]):
        Foata[i].sort(key=lambda y: vertexes[y])
        print("[", end=' ')
        for v in Foata[i]:
            if not more:
                print(f"{ind_char(vertexes[v])}", end=' ')
            else:
                print(vertexes[v], end = ' ')
        print("]", end=' ')
    else:
        break
f1.close()
f2.close()
Gr.render(".\\Graph", view = True)

26 0 39 23 14 20 43 49 42 45 27 43 35 5 36 53 21 52 58 17 48 58 4 61 
Foata classes:
[ 0 26 ] [ 39 ] [ 23 ] [ 14 20 49 ] [ 42 43 45 ] [ 27 ] [ 35 43 53 ] [ 5 52 61 ] [ 36 ] [ 21 48 58 ] [ 4 17 58 ] 

'Graph.png'